In [1]:
import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')

In [2]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 8773482321084670535, name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 9223152599
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 7899180661387135687
 physical_device_desc: "device: 0, name: GeForce GTX 1080 Ti, pci bus id: 0000:01:00.0, compute capability: 6.1"]

In [3]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials, partial, rand, space_eval
from sklearn.metrics import log_loss
import sys

import os
import numpy as np

import keras
from keras.optimizers import Adam, RMSprop,SGD
from keras.models import Model, Sequential
from keras.layers import Flatten, Dense, Activation, Dropout,Input, concatenate, Flatten  
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping
from keras.applications.vgg19 import VGG19
from keras import optimizers, metrics, models, layers
from keras.utils.np_utils import to_categorical
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, auc

import matplotlib.pyplot as plt
%matplotlib inline

Using TensorFlow backend.


In [4]:
import keras
import tensorflow as tf
import keras.backend as K

print("Keras Version", keras.__version__)
print("tensorflow Version", tf.__version__)
print("dim_ordering:", K.image_dim_ordering())

Keras Version 2.1.6
tensorflow Version 1.10.0
dim_ordering: tf


In [5]:
train_dir = 'data/ISIC2017/train/'
test_dir = 'data/ISIC2017/test/'

extracted_features_dir = "extracted_features/"
num_classes = 2
batch_size = 32
img_height, img_width = 512, 512
input_shape = (img_height, img_width, 3)
epochs = 15

nb_train_samples = sum([len(files) for r, d, files in os.walk(train_dir)])
nb_test_samples = sum([len(files) for r, d, files in os.walk(test_dir)])

In [6]:
random_seed = np.random.seed(1142)

train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    featurewise_center=True,
    featurewise_std_normalization=True,
    validation_split= 0.25,
    zoom_range=0.2,
    shear_range=0.2)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    seed = random_seed,
    shuffle = False,
    subset = 'training',
    class_mode='categorical')

validation_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    seed = random_seed,
    shuffle = False,
    subset = 'validation',
    class_mode='categorical')


test_datagen = ImageDataGenerator(rescale=1. / 255)
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    seed = random_seed,
    shuffle = False,
    class_mode='categorical')

Found 24373 images belonging to 2 classes.
Found 8124 images belonging to 2 classes.
Found 600 images belonging to 2 classes.


In [7]:
train_data = np.load(extracted_features_dir+'bottleneck_features_train_VGG19_2017.npy')
validation_data = np.load(extracted_features_dir+'bottleneck_features_validation_VGG19_2017.npy')
test_data = np.load(extracted_features_dir+'bottleneck_features_test_VGG19_2017.npy')

train_labels = train_generator.classes
train_labels = to_categorical(train_labels, num_classes=num_classes)

validation_labels = validation_generator.classes
validation_labels = to_categorical(validation_labels, num_classes=num_classes)

test_labels = test_generator.classes
test_labels = to_categorical(test_labels, num_classes=num_classes)

In [8]:
space = {
         'lr': hp.choice('lr',[0.001, 0.0001, 0.00001,0.000001, 0.0000001]),
         'dropout': hp.choice('dropout', [0.4, 0.5, 0.6, 0.7]),
#          'batch_size': hp.choice('batch_size', [64]),
#          'epochs': hp.choice('epochs', [15, 20, 25, 30, 50]),
#          'optimizer': hp.choice('optimizer',['sgd','adam','rmsprop']),
#          'optimizer': hp.choice('optimizer',['rmsprop']),
#          'optimizer': hp.choice('optimizer',['adam']),
         'beta_1':hp.choice('beta_1',[0.3,0.4,0.5,0.6,0.7,0.8]),
         'beta_2':hp.choice('beta_2',[0.99,0.995,0.7,0.8,0.9,0.999]),
#          'momentum':hp.choice('momentum',[0.3,0.5,0.7,0.9,1]),
         'amsgrad':hp.choice('amsgrad',[False,True]),
#          'nesterov':hp.choice('nesterov',[False,True]),
#          'rho':hp.choice('rho',[0.4,0.5,0.6,0.7,0.8,0.9,1]),
        }

In [10]:
def f_nn(params):   
    print ('Parameters testing: ', params)
#     batch_size=params['batch_size']

    model = Sequential()
    model.add(Flatten(input_shape=train_data.shape[1:]))
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(params['dropout']))
    model.add(Dense(num_classes, activation='softmax'))

   
    adam_opt=Adam(lr=params["lr"], beta_1=params["beta_1"], beta_2=params['beta_2'], epsilon=None, decay=0.0, amsgrad=params['amsgrad'])
#     sgd=SGD(lr=params["lr"], momentum=params['momentum'], decay=0.0, nesterov=params['nesterov'])
#     rmsprop=RMSprop(lr=params["lr"], rho=params['rho'], epsilon=None, decay=0.0)

    
    model.compile(optimizer = adam_opt, loss='categorical_crossentropy', metrics=['accuracy'])

    history = model.fit(train_data, train_labels,
                    epochs=epochs,
                    batch_size=batch_size,
                    validation_data=(validation_data, validation_labels),
                    verbose= 2,
                    callbacks=get_callbacks(params))
    
    (eval_loss, eval_accuracy) = model.evaluate(validation_data, validation_labels, batch_size= batch_size, verbose=1)

    print("Validation Accuracy: {:.4f}%".format(eval_accuracy * 100))
    print("Validation Loss: {}".format(eval_loss))
    
    print("_" * 30)
    preds = model.predict(test_data)

    predictions = [i.argmax() for i in preds]
    y_true = [i.argmax() for i in test_labels]

    print('Test Accuracy {}'.format(accuracy_score(y_true=y_true, y_pred=predictions)))
    print("_" * 50)
    sys.stdout.flush() 
    keras.backend.clear_session()
    
    return {'loss': eval_loss, 'status': STATUS_OK, 'model': model}

In [11]:
def get_callbacks(params):
    callbacks =[EarlyStopping(monitor='val_loss', patience=3, verbose=1)]
    return callbacks

In [12]:
trials = Trials()
best = fmin(f_nn, space, algo=tpe.suggest, max_evals=30, trials=trials)
print(best)

Parameters testing:  {'amsgrad': True, 'beta_1': 0.8, 'beta_2': 0.9, 'dropout': 0.5, 'lr': 0.0001}
Train on 24373 samples, validate on 8124 samples
Epoch 1/15
 - 32s - loss: 0.6772 - acc: 0.6017 - val_loss: 0.6335 - val_acc: 0.6403
Epoch 2/15
 - 28s - loss: 0.6391 - acc: 0.6202 - val_loss: 0.6140 - val_acc: 0.6773
Epoch 3/15
 - 28s - loss: 0.6194 - acc: 0.6306 - val_loss: 0.6015 - val_acc: 0.7069
Epoch 4/15
 - 28s - loss: 0.6036 - acc: 0.6434 - val_loss: 0.5846 - val_acc: 0.6507
Epoch 5/15
 - 28s - loss: 0.5888 - acc: 0.6632 - val_loss: 0.5620 - val_acc: 0.6999
Epoch 6/15
 - 29s - loss: 0.5784 - acc: 0.6829 - val_loss: 0.5580 - val_acc: 0.7340
Epoch 7/15
 - 28s - loss: 0.5642 - acc: 0.6936 - val_loss: 0.5569 - val_acc: 0.6896
Epoch 8/15
 - 28s - loss: 0.5553 - acc: 0.6997 - val_loss: 0.5270 - val_acc: 0.7402
Epoch 9/15
 - 28s - loss: 0.5454 - acc: 0.7090 - val_loss: 0.5430 - val_acc: 0.7534
Epoch 10/15
 - 29s - loss: 0.5394 - acc: 0.7136 - val_loss: 0.5130 - val_acc: 0.7366
Epoch 11/15

Parameters testing:  {'amsgrad': False, 'beta_1': 0.5, 'beta_2': 0.8, 'dropout': 0.7, 'lr': 1e-07}
Train on 24373 samples, validate on 8124 samples
Epoch 1/15
 - 28s - loss: 0.8843 - acc: 0.5396 - val_loss: 0.6630 - val_acc: 0.6034
Epoch 2/15
 - 26s - loss: 0.7668 - acc: 0.5488 - val_loss: 0.6568 - val_acc: 0.6057
Epoch 3/15
 - 26s - loss: 0.7121 - acc: 0.5599 - val_loss: 0.6538 - val_acc: 0.6052
Epoch 4/15
 - 26s - loss: 0.6836 - acc: 0.5832 - val_loss: 0.6502 - val_acc: 0.6087
Epoch 5/15
 - 26s - loss: 0.6688 - acc: 0.5962 - val_loss: 0.6482 - val_acc: 0.6114
Epoch 6/15
 - 26s - loss: 0.6629 - acc: 0.6049 - val_loss: 0.6467 - val_acc: 0.6102
Epoch 7/15
 - 27s - loss: 0.6597 - acc: 0.6103 - val_loss: 0.6449 - val_acc: 0.6164
Epoch 8/15
 - 26s - loss: 0.6591 - acc: 0.6139 - val_loss: 0.6436 - val_acc: 0.6185
Epoch 9/15
 - 26s - loss: 0.6552 - acc: 0.6145 - val_loss: 0.6423 - val_acc: 0.6262
Epoch 10/15
 - 26s - loss: 0.6535 - acc: 0.6150 - val_loss: 0.6408 - val_acc: 0.6258
Epoch 11/15

Parameters testing:  {'amsgrad': True, 'beta_1': 0.6, 'beta_2': 0.999, 'dropout': 0.4, 'lr': 0.001}
Train on 24373 samples, validate on 8124 samples
Epoch 1/15
 - 30s - loss: 6.5079 - acc: 0.5952 - val_loss: 9.7058 - val_acc: 0.3978
Epoch 2/15
 - 28s - loss: 6.7763 - acc: 0.5792 - val_loss: 6.4123 - val_acc: 0.6022
Epoch 3/15
 - 29s - loss: 6.4125 - acc: 0.6021 - val_loss: 6.4123 - val_acc: 0.6022
Epoch 4/15
 - 29s - loss: 6.4124 - acc: 0.6021 - val_loss: 6.4123 - val_acc: 0.6022
Epoch 5/15
 - 28s - loss: 6.4121 - acc: 0.6022 - val_loss: 6.4123 - val_acc: 0.6022
Epoch 00005: early stopping
8124/8124 [==============================] - 4s 452us/step
Validation Accuracy: 60.2166%
Validation Loss: 6.412319658954175
______________________________
Test Accuracy 0.805
__________________________________________________
Parameters testing:  {'amsgrad': False, 'beta_1': 0.5, 'beta_2': 0.995, 'dropout': 0.5, 'lr': 1e-06}
Train on 24373 samples, validate on 8124 samples
Epoch 1/15
 - 28s - loss: 0

Epoch 8/15
 - 28s - loss: 0.5460 - acc: 0.7064 - val_loss: 0.5307 - val_acc: 0.7469
Epoch 9/15
 - 28s - loss: 0.5344 - acc: 0.7208 - val_loss: 0.5113 - val_acc: 0.7523
Epoch 10/15
 - 28s - loss: 0.5262 - acc: 0.7265 - val_loss: 0.5130 - val_acc: 0.7473
Epoch 11/15
 - 28s - loss: 0.5157 - acc: 0.7341 - val_loss: 0.5098 - val_acc: 0.7275
Epoch 12/15
 - 28s - loss: 0.5109 - acc: 0.7363 - val_loss: 0.4946 - val_acc: 0.7607
Epoch 13/15
 - 28s - loss: 0.5035 - acc: 0.7437 - val_loss: 0.4900 - val_acc: 0.7653
Epoch 14/15
 - 28s - loss: 0.4935 - acc: 0.7505 - val_loss: 0.4933 - val_acc: 0.7682
Epoch 15/15
 - 28s - loss: 0.4844 - acc: 0.7546 - val_loss: 0.4849 - val_acc: 0.7574
8124/8124 [==============================] - 3s 423us/step
Validation Accuracy: 75.7386%
Validation Loss: 0.4849165275377455
______________________________
Test Accuracy 0.7883333333333333
__________________________________________________
Parameters testing:  {'amsgrad': False, 'beta_1': 0.7, 'beta_2': 0.7, 'dropout': 0

Parameters testing:  {'amsgrad': True, 'beta_1': 0.8, 'beta_2': 0.9, 'dropout': 0.5, 'lr': 1e-05}
Train on 24373 samples, validate on 8124 samples
Epoch 1/15
 - 31s - loss: 0.6602 - acc: 0.6090 - val_loss: 0.6242 - val_acc: 0.6556
Epoch 2/15
 - 28s - loss: 0.6260 - acc: 0.6429 - val_loss: 0.6543 - val_acc: 0.6089
Epoch 3/15
 - 29s - loss: 0.6053 - acc: 0.6673 - val_loss: 0.5790 - val_acc: 0.6817
Epoch 4/15
 - 29s - loss: 0.5861 - acc: 0.6824 - val_loss: 0.5607 - val_acc: 0.7224
Epoch 5/15
 - 28s - loss: 0.5722 - acc: 0.6936 - val_loss: 0.5470 - val_acc: 0.7325
Epoch 6/15
 - 28s - loss: 0.5581 - acc: 0.7081 - val_loss: 0.5383 - val_acc: 0.7267
Epoch 7/15
 - 29s - loss: 0.5469 - acc: 0.7172 - val_loss: 0.5765 - val_acc: 0.6603
Epoch 8/15
 - 29s - loss: 0.5360 - acc: 0.7260 - val_loss: 0.5251 - val_acc: 0.7473
Epoch 9/15
 - 29s - loss: 0.5266 - acc: 0.7318 - val_loss: 0.5166 - val_acc: 0.7457
Epoch 10/15
 - 28s - loss: 0.5190 - acc: 0.7371 - val_loss: 0.5148 - val_acc: 0.7563
Epoch 11/15


Parameters testing:  {'amsgrad': True, 'beta_1': 0.8, 'beta_2': 0.9, 'dropout': 0.5, 'lr': 1e-05}
Train on 24373 samples, validate on 8124 samples
Epoch 1/15
 - 30s - loss: 0.6608 - acc: 0.6088 - val_loss: 0.6267 - val_acc: 0.6300
Epoch 2/15
 - 28s - loss: 0.6294 - acc: 0.6411 - val_loss: 0.6059 - val_acc: 0.6885
Epoch 3/15
 - 28s - loss: 0.6054 - acc: 0.6670 - val_loss: 0.6056 - val_acc: 0.6342
Epoch 4/15
 - 28s - loss: 0.5900 - acc: 0.6790 - val_loss: 0.5896 - val_acc: 0.6530
Epoch 5/15
 - 28s - loss: 0.5728 - acc: 0.6913 - val_loss: 0.5493 - val_acc: 0.7150
Epoch 6/15
 - 28s - loss: 0.5592 - acc: 0.7067 - val_loss: 0.5408 - val_acc: 0.7346
Epoch 7/15
 - 28s - loss: 0.5445 - acc: 0.7201 - val_loss: 0.5391 - val_acc: 0.7390
Epoch 8/15
 - 28s - loss: 0.5364 - acc: 0.7251 - val_loss: 0.5242 - val_acc: 0.7317
Epoch 9/15
 - 28s - loss: 0.5271 - acc: 0.7289 - val_loss: 0.5217 - val_acc: 0.7528
Epoch 10/15
 - 28s - loss: 0.5164 - acc: 0.7434 - val_loss: 0.5178 - val_acc: 0.7218
Epoch 11/15
